# Presentation Notebook CFS Capital
Thursday September 20th <br>
<br>
    Eric Brea Garcia
    <br>
    Diederik Ketellapper
    <br>
    Viktor Malesevic
    <br>
    Daniel Wientjens

#### Description

The goal of the project is to predict whether the bitcoin price is going up or down. For this project we use data from the 1st of May 2017 untill the first of May 2018.

We use a variety of sources of information each on a **daily level**:
1. All the reddit comments that reference 'bitcoin' for this timeperiod utilizing google bigquery
2. Forum Data from bis
3. Google Trends data where we retrieve the bitcoin buy/ bitcoin sell +bitcoin buy fraction
4. Tweets by the 100 biggest bitcoin influencers of this time period
5. Bitcoin actual data from poloniex

After this exercise we need to combine the data together to form a master table and then divide this into: Train,Validation and Test.

As a penultimate step we use H2O automl to train a variety of models and use an ensemble to train and predict.

Then finally we report on our data, what the return of our strategy would be for March and April of 2018.

_Please note that not all code of this notebook runs, and it should not be used as such because of the long datadownloads, but it is intended to give a short concise overview of our project and the results._


# 1. Reddit Comments

We downloaded all the reddit comments in the given time period and afterwards performed sentimed analysis and grouped by day

We used google cloud for the processing power to download all the reddit comments referencing bitcoin within the time period as below

#standardSQL
SELECT created_utc, body

FROM(
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_05` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_06` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_07` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_08` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_09` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_10` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_11` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2017_12` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2018_01` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2018_02` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2018_03` UNION ALL
  SELECT
    *
  FROM
    `fh-bigquery.reddit_comments.2018_04`) 
    
WHERE REGEXP_CONTAINS(body, r'bitcoin')

In [1]:
import pandas as pd
import re
from datetime import datetime
import re

In [2]:
# download the reddit comments
reddit = pd.read_csv("/Users/Mobile/Downloads/r_comments")

In [ ]:
# define a cleaning function for the reddit comments
def clean(body):
 return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", body).split())

In [ ]:
# clean the reddit comments and dates
reddit['body'] = reddit.body.apply(clean)
reddit['date'] = reddit.created_utc.apply(datetime.fromtimestamp)
reddit['date'] = reddit.date.dt.date

In [ ]:
# Here we perform the sentiment analysis it already computes the magnitude times the sentiment

# Import the module
from textblob import TextBlob
def analize_sentiment(comment):
    analysis = TextBlob(comment)
    return(analysis.sentiment.polarity)

reddit['SA'] = np.array([ analize_sentiment(comment) for comment in reddit['body']])

Here we have to group the data by day and we do that by looking at the count of comments and the average sentiment analysis. 

In [ ]:
# group by date
reddit_groupby= reddit.groupby('date').agg({'body':'count', 'SA':'mean'})

In [ ]:
reddit_groupby = reddit_groupby.rename(columns={'body':'count_comments', 'SA':'mean_sa'}).reset_index()

In [ ]:
reddit_groupby.to_csv("Data/allreddit_nlp.csv")

# 2 Forum Data

 The forum data is a similiar exercise to the reddit comments and is here ommited for brevity 

# 3 Google Trends Data 

Here we download data from Google trends and carry out some of the pre-processing related to it. What we look at is the ratio of "buy bitcoin" versus the sum of "buy bitcoin" & "sell bitcoin" to get a feeling about how the trend is evolving over time. This solves as well one of our problems as google does not output the raw nummer of searches but gives you a number representing the change in your time period. Multiple downloads over multiple timeperiods cannot be compared as it seems that google adds some noise as well. 
This fraction is a solution but we do not assume it is the most optimal one.

In [ ]:
# necessary imports
from pytrends.request import TrendReq

In [ ]:
# Get the relevant timeframes to search on
timeframes2017 = list()
timeframesmix = list()
timeframes2018 = list()
for i in range(5,11):
    timeframes2017.append('2017-'+str(i)+'-1 2017-'+str(i+2)+'-1')
timeframesmix.append('2017-'+'11'+'-1 2018-'+'1'+'-1')
timeframesmix.append('2017-'+'12'+'-1 2018-'+'2'+'-1')
for i in range(1,5):
    timeframes2018.append('2018-'+str(i)+'-1 2018-'+str(i+2)+'-1')
timeframes = timeframes2017+timeframesmix+timeframes2018

In [ ]:
# download using pytrends the referenced timeperiods
pytrend = TrendReq(hl='en-US', tz=360)
list_google = list()
for timeframe in timeframes:
    pytrend.build_payload(kw_list=['Buy Bitcoin', 'Sell Bitcoin'], 
                      timeframe=timeframe)
    list_google.append(pytrend.interest_over_time())

In [ ]:
# append all the dataframes to one large one
df = pd.DataFrame(list_google[0])

for i in range(1,11):
    df = df.append(list_google[i])

In [ ]:
# Calculate the fraction
df['googletrends_buy_sell'] = df["Buy Bitcoin"]/(df["Sell Bitcoin"]+df["Buy Bitcoin"])
df = df[['googletrends_buy_sell']] 

In [ ]:
# take the mean for the overlapping dates and output the data
df = df.groupby('date').mean().reset_index()
df.to_csv("Data/google_trends.csv")

# 4 Downloading the Twitter Dataset of the 100 biggest influencers

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup

def getTwitterHandles():
    # Fill in with url of page which is to be scraped
    url = "https://cryptoweekly.co/100/"

# Retreives and parses page html
    client = urlopen(url)
    pageHtml = client.read()
    pageSoup = soup(pageHtml, "html.parser")

# Adds all Twitter handles to twitterHandles list
    profiles = pageSoup.findAll("div", {"class":"testimonial-wrapper"})
    twitterHandles = []
    for person in profiles:
        twitterHandles.append(person.findAll("div",{"class":"author"}))
    for i in range(len(twitterHandles)):
        twitterHandles[i]=twitterHandles[i][0].findAll("a")[0].text[1:]

    client.close()
    return twitterHandles

Below we download the tweets. Because of the large download please consider carefully to run

In [ ]:
import tweepy
import csv
import sys

# Twitter API credentials (expired, don't even try it)
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
	print("Getting tweets from @" + str(screen_name))

	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print ("Getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print ("...%s tweets downloaded so far" % (len(alltweets)))

	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str.encode('utf-8'), tweet.created_at.strftime('%m/%d/%Y'), tweet.text.encode('utf-8')] for tweet in alltweets]
	
	#write the csv	
	with open('Data/Tweets/%s_tweets.csv' % screen_name, 'w') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
	
	pass


if __name__ == '__main__':
	handles = getTwitterHandles()
    # The Peter Todd twitter handle is updated to his new account, as long as the old one was listed in the website:
	for i in range(len(handles)):
		if handles[i] == 'petertoddbtc':
			handles[i] = 'peterktodd'
	for handle in handles:
		get_all_tweets(str(handle))

After downloading the tweets we do a similar exercise as with the reddit comments.
We also remove any tweets that are not in the correct timeframe. This is ommited for brevity.

# 5 Importing the poloniex data 

## Step 1. Importing from Poloniex
Poloniex is a cryptocurrency exchange found on https://poloniex.com/exchange, to download the data we use a package made by a poloniex community member.

In [6]:
import poloniex
import datetime

Importing from API

To import poloniex we need to install the package in the console using:
pip install poloniex

The help function gives a list of functions included in the package and some descriptions
help(poloniex.poloniex)

We are using the public data so no keys are needed

The package has many functionalities but the information we want is public, therefore we do not need access tokens or secret keys. The currency we are looking for is BitCoin relative to the USD, therefore looking at the documentation we know that we need to search using the term 'USDT_BTC', with an additional parameter of seconds between measurements. We chose to take the 24 hours frequency.

In [8]:
polo = poloniex.Poloniex()
btc86400 = pd.DataFrame(polo.returnChartData("USDT_BTC", 86400))

## Step 2. Pre-processing


In [9]:
#Transform the unix timestamp to datetime:
btc86400.date = pd.to_datetime(btc86400.date, unit = 's')
#Setting the date as index:
btc86400.set_index('date', inplace = True)
#Taking the period 2017-05-01 to 2018-05-02 (one more day for lagged purposes):
btc_data = btc86400["2017-04-16":"2018-05-02"]
#We decide to take only close and volume since the other columns are extremly correlated to these and may not provide more useful information for our purpose.
returns_data = btc_data[["close"]]
#Now let's look at the returns (indeed, the close price itself is not particularly interesting):
returns_data = returns_data.pct_change()
volatility_data = returns_data.rolling(14).std()
#We are going to add the volatility of the last 14 days:
volatility_data = volatility_data.rename(columns = {'close': 'volatility_14'})

In [10]:
# final changes for clarity
returns_data = returns_data.shift(-1)
returns_data = returns_data.dropna()
returns_data.columns = ["return_day+1"]

In [ ]:
#Now we put back together the returns at day+1 with the close and volume information from earlier. We also dropna() for deleting the day 2018-05-02.
data = pd.concat((returns_data, btc_data[["close","volume"]]), axis = 1)
data = pd.concat((data, volatility_data), axis = 1)
data = data.dropna()
# filter
data = data["may-2017":"may-2018"]
data.to_csv("Data/poloniex_data.csv")
#Creating a csv file for later purposes and checking that reading it works.

# Combining data

In [ ]:
# Load CSV files:
poloniex = pd.read_csv("Data/poloniex_data.csv")
ggtrends = pd.read_csv("Data/google_trends.csv")
twitter = pd.read_csv("Data/twitter_agg_ddb.csv")
reddit = pd.read_csv("Data/allreddit_nlp.csv")
forum = pd.read_csv("Data/merit_compound.csv")

In [ ]:
# A bit of preprocessing:
# GOOGLE TRENDS:
ggtrends = ggtrends.loc[:, ~ggtrends.columns.str.contains('^Unnamed')]
# REDDIT:
reddit = reddit.loc[:, ~reddit.columns.str.contains('^Unnamed')]
reddit = reddit.rename(columns={'date_notime': 'date', 'count_comments': 'Reddit Comments (#)', 'mean_sa': 'Reddit Average SA'})
# TWITTER:
twitter = twitter.rename(columns={'created_at': 'date','Average SA': 'Twitter Average SA'})
twitter['date'] =  pd.to_datetime(twitter['date'], format='%Y%m%d %H:%M:%S')
twitter['date'] = twitter['date'].dt.date
twitter['date'] = twitter['date'].apply(str)

# FORUM:
forum = forum.loc[:, ~forum.columns.str.contains('^Unnamed')]
forum = forum.rename(columns={'newdate': 'date', 'compound': 'Forum SA Merit', 'merit_compound': 'Forum SA Merit (weighted)'})

In [ ]:
# Merge the datasets into one Master Table:
master = pd.merge(poloniex, ggtrends, how = 'inner', on = 'date')
master = pd.merge(master, twitter, how = 'outer', on = 'date')
master = pd.merge(master, forum, how = 'inner', on = 'date')
master = pd.merge(master, reddit, how = 'inner', on = 'date')

In [ ]:
# Create the UP/DOWN class:
master['invest'] = master['return_day+1']>0